In [1]:
import os
import re
import shutil
from pathlib import Path

DEPENDENCIES_SAVE_PATH = Path('dependencies')
SOURCE_CODE_SAVE_PATH = Path('source-code')
SOURCE_CODE_PATH = Path('../')
#pip install -r C://Users//Tolga//Desktop//EPOCH-IV//q3-harmful-brain-activity//submission//dependencies//requirements.txt --no-index --find-links=file://C://Users//Tolga//Desktop//EPOCH-IV//q3-harmful-brain-activity//submission//dependencies//tmp
UPDATE_DEPENDENCIES = False
UPDATE_SOURCE_CODE = True

# Manages Depenedencies

### Download Dependencies and ZIP them

In [2]:
if UPDATE_DEPENDENCIES:
    if os.path.exists(DEPENDENCIES_SAVE_PATH):
        print('Cleaning the dependencies folder')
        for filename in os.listdir(DEPENDENCIES_SAVE_PATH):
            file_path = os.path.join(DEPENDENCIES_SAVE_PATH, filename)
            if filename != 'tmp':
                if os.path.isfile(file_path):
                    os.remove(file_path)
                elif os.path.isdir(file_path):
                    shutil.rmtree(file_path)
    else:
        os.makedirs(DEPENDENCIES_SAVE_PATH)

    print('Copying the requirements.txt file and excluding -e')
    with open(SOURCE_CODE_PATH / 'requirements.txt', 'r') as f:
        lines = f.readlines()
    with open(DEPENDENCIES_SAVE_PATH / 'requirements.txt', 'w') as f:
        for line in lines:
            if line.startswith('-e'):
                continue
            if line.startswith('kaggle'):
                continue
            f.write(line)

    if not os.path.exists(DEPENDENCIES_SAVE_PATH / 'tmp'):
        os.makedirs(DEPENDENCIES_SAVE_PATH / 'tmp')
    print('Downloading the dependencies')
    if not os.path.exists(DEPENDENCIES_SAVE_PATH / 'tmp'):
        os.makedirs(DEPENDENCIES_SAVE_PATH / 'tmp')
    !pip download -r {DEPENDENCIES_SAVE_PATH / 'requirements.txt'} -d {DEPENDENCIES_SAVE_PATH / 'tmp'}

    print('Zipping the downloaded dependencies')
    shutil.make_archive(DEPENDENCIES_SAVE_PATH / 'dependencies', 'zip', DEPENDENCIES_SAVE_PATH / 'tmp')
    shutil.move(DEPENDENCIES_SAVE_PATH / 'dependencies.zip', DEPENDENCIES_SAVE_PATH / 'dependencies.no_unzip')
    shutil.rmtree(DEPENDENCIES_SAVE_PATH / 'tmp')

    print('Copying the dataset-metadata.json file')
    shutil.copy('dataset-metadata-dependencies.json', DEPENDENCIES_SAVE_PATH / 'dataset-metadata.json')

    print('Excluding --find-files in requirements.txt')
    with open(DEPENDENCIES_SAVE_PATH / 'requirements.txt', 'r') as f:
        lines = f.readlines()
    with open(DEPENDENCIES_SAVE_PATH / 'requirements.txt', 'w') as f:
        for line in lines:
            if line.startswith('--find-links'):
                continue
            f.write(line)

    print('Done')

Cleaning the dependencies folder
Copying the requirements.txt file and excluding -e
Looking in links: https://download.pytorch.org/whl/torch_stable.html
  Using cached https://download.pytorch.org/whl/rocm5.7/torch-2.2.1%2Brocm5.7-cp311-cp311-linux_x86_64.whl (1685.3 MB)
  Using cached https://download.pytorch.org/whl/rocm5.7/torchaudio-2.2.1%2Brocm5.7-cp311-cp311-linux_x86_64.whl (1.7 MB)
  Using cached https://download.pytorch.org/whl/rocm5.7/torchvision-0.17.1%2Brocm5.7-cp311-cp311-linux_x86_64.whl (65.5 MB)
  Using cached annotated_types-0.6.0-py3-none-any.whl.metadata (12 kB)
  Using cached coloredlogs-15.0.1-py2.py3-none-any.whl.metadata (12 kB)
  Using cached dask_expr-1.0.4-py3-none-any.whl.metadata (2.4 kB)
  Using cached epochalyst-0.1.6-py3-none-any.whl.metadata (2.2 kB)
  Using cached hydra_core-1.3.2-py3-none-any.whl.metadata (5.5 kB)
  Using cached numpy-1.26.4-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
  Using cached pyarrow-15.0.2-cp311-

### Upload Dependencies to Kaggle as a Dataset

In [3]:
if UPDATE_DEPENDENCIES:
    # !kaggle datasets create -p ./dependencies
    !kaggle datasets version -p ./dependencies -m "Update Dependencies"


Starting upload for file requirements.txt
100%|████████████████████████████████████████████| 338/338 [00:00<00:00, 531B/s]
Upload successful: requirements.txt (338B)
Starting upload for file dependencies.no_unzip
100%|██████████████████████████████████████| 1.97G/1.97G [00:51<00:00, 40.6MB/s]
Upload successful: dependencies.no_unzip (2GB)
Dataset version is being created. Please check progress at https://www.kaggle.com/justanotherariel/epoch-hms-dependencies


# Manage Source Code

### Copy Source Code and ZIP it

In [4]:
if UPDATE_SOURCE_CODE:
    if os.path.exists(SOURCE_CODE_SAVE_PATH):
        shutil.rmtree(SOURCE_CODE_SAVE_PATH)
    os.mkdir(SOURCE_CODE_SAVE_PATH)

    # Copy Source Code to submission/source_code
    relevant_files = ['src/', 'conf/', 'tm/', 'submit.py']
    exluded_files = ['__pycache__']
    for file in relevant_files:
        if os.path.isdir(SOURCE_CODE_PATH / file):
            # Copy directory, skip excluded files with shutil
            shutil.copytree(SOURCE_CODE_PATH / file, SOURCE_CODE_SAVE_PATH / "tmp" / file, ignore=shutil.ignore_patterns(*exluded_files))
        else:
            # Copy file
            shutil.copy(SOURCE_CODE_PATH / file, SOURCE_CODE_SAVE_PATH / "tmp" / file)

    # Zip source_code
    shutil.make_archive(SOURCE_CODE_SAVE_PATH / 'source-code', 'zip', SOURCE_CODE_SAVE_PATH / "tmp")
    shutil.rmtree(SOURCE_CODE_SAVE_PATH / "tmp")

    # Copy dataset-metadata.json to submission
    shutil.copy('dataset-metadata-source-code.json', SOURCE_CODE_SAVE_PATH / 'dataset-metadata.json')

    print('Submission files saved to source_code')

Submission files saved to source_code


### Upload Source Code

In [5]:
if UPDATE_SOURCE_CODE:
    # !kaggle datasets create -p ./source-code
    !kaggle datasets version -p ./source-code -m "Update Source Code"

Starting upload for file source-code.zip
100%|██████████████████████████████████████| 4.40M/4.40M [00:01<00:00, 3.02MB/s]
Upload successful: source-code.zip (4MB)
Dataset version is being created. Please check progress at https://www.kaggle.com/justanotherariel/epoch-hms-source-code
